In [1]:
# Randomly pick the values of S, X, R in different time

import numpy as np
n = 6000
S = np.random.randint(0.5*n, 0.8*n, size=30)
S_array = np.sort(S)[::-1]
remaining = n*np.ones(30) - S_array
R_array = np.zeros(30)

for r in range(30):
    R_array[r] = np.random.randint(R_array[r-1]+1, n-S_array[r])

X_array = remaining - R_array

In [2]:
# Calculate the values of beta and gamma

beta_array = -n * np.diff(S_array) / (S_array[:-1] * X_array[:-1])
gamma_array = np.diff(R_array) / X_array[:-1]

In [7]:
# Calculate the values of coefficients

beta_results = beta_array[14:]
beta_vals = np.array([np.append(beta_array[r:r+14][::-1],1) for r in range(15)])
a_seq = np.linalg.inv(beta_vals) @ beta_results

gamma_results = gamma_array[14:]
gamma_vals = np.array([np.append(gamma_array[r:r+14][::-1],1) for r in range(15)])
b_seq = np.linalg.inv(gamma_vals) @ gamma_results

In [10]:
# Calculate and predict 50 R(t) with SIR model

reproduce_0 = beta_array / gamma_array
S_pred = S_array[-1]
X_pred = X_array[-1]
R_pred = R_array[-1]

reproduce_1 = np.zeros(5)
for r in range(5):
    beta_pred = np.dot(a_seq, np.append(beta_array[15+r:][::-1],1))
    beta_array = np.append(beta_array, beta_pred)
    S_pred = - beta_pred * S_pred * X_pred / n + S_pred

    gamma_pred = np.dot(b_seq, np.append(gamma_array[15+r:][::-1],1))
    gamma_array = np.append(gamma_array, gamma_pred)
    R_pred = gamma_pred * X_pred + R_pred

    X_pred = n - S_pred - R_pred

    reproduce_1[r] = beta_pred / gamma_pred

4963.857456206499
3002.463261905444
-1966.3207181119428
246394.5344676056
18199.027008863606
-258593.5614764692
2129736776.146463
-11606829.06667155
-2118123947.0797913
-5.927439779406244e+17
458672099150.342
5.927435192685312e+17
2.432305969026292e+34
6.444542741765001e+20
-2.4323059690263567e+34


In [11]:
# Calculate the values of I(t)
I_array = -np.diff(S_array)

# solve the values of w(s)
w_array = np.zeros(28)
for r in range(1,29):
    w_array[r-1] =(I_array[r] / reproduce_0[r] - np.dot(I_array[r-1:0:-1], w_array[:r-1])) / I_array[0]

# predict 50 R(t) by w(s) and TSI model
